# import


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.csgraph import floyd_warshall
import math
import urllib.request
import io
import zipfile
import json

# function


## get data 

https://networkx.org/documentation/stable/auto_examples/graph/plot_football.html#sphx-glr-auto-examples-graph-plot-football-py

In [ ]:
def get_football_graph_data():
    url = "http://www-personal.umich.edu/~mejn/netdata/football.zip"

    sock = urllib.request.urlopen(url)  # open URL
    s = io.BytesIO(sock.read())  # read into BytesIO "file"
    sock.close()

    zf = zipfile.ZipFile(s)  # zipfile object
    txt = zf.read("football.txt").decode()  # read info file
    gml = zf.read("football.gml").decode()  # read gml data
    # throw away bogus first line with # from mejn files
    gml = gml.split("\n")[1:]
    G = nx.parse_gml(gml)  # parse gml data
    print(len(G))
    return G

https://ialab.it.monash.edu/webcola/examples/unconstrained.html

In [ ]:
def get_webcola_graph_data():
    data = None
    with open("./data.json") as f:
        data = json.load(f)
    node_data = [i for i, _ in enumerate(data["nodes"])]
    edge_data = [[d["target"], d["source"]] for d in data["links"]]
    G = nx.Graph()
    G.add_nodes_from(node_data)
    G.add_edges_from(edge_data)
    return G

## util

In [ ]:
def adjacency_matrix(G):
    A = np.zeros((len(G), len(G)))
    for i, ni in enumerate(G.nodes()):
        for j, nj in enumerate(G.nodes()):
            if G.has_edge(ni, nj):
                A[i][j] = 1
    return A

In [ ]:
def plot_graph(
    G,
    P,
    options={"node_color": "black", "node_size": 50, "linewidths": 0, "width": 0.1},
):
    nx.draw(G, P, **options)
    plt.show()

In [ ]:
def get_graph_property(G):
    # layout_pos = nx.random_layout(G)
    # P = np.array(list(layout_pos.values()))
    P = nx.random_layout(G)
    D = floyd_warshall(adjacency_matrix(G), directed=False)
    W = np.array(
        [
            [1 / (dij * dij) if not dij == 0 and not dij == 0 else 0 for dij in di]
            for di in D
        ]
    )

    return P, D, W

In [ ]:
def stress(P, D, W):
    stress_sum = 0
    for i, p in enumerate(P.values()):
        for j, q in enumerate(P.values()):
            stress_sum += W[i][j] * (
                (math.dist(p, q) - D[i][j]) * (math.dist(p, q) - D[i][j])
            )
    return stress_sum

# vis

## stress only


In [ ]:
def plot_spring_layout(G):
    options={"node_color": "black", "node_size": 50, "linewidths": 0, "width": 0.1}
    P = nx.spring_layout(G)
    nx.draw(G, P, **options)
    plt.show()

In [ ]:
def next_position(u, W, D, P):
    weight_sum = sum([w for w in W[u]])
    vu = P[u]
    x_sum = 0
    y_sum = 0
    for i, w in enumerate(W[u]):
        if i == u:
            continue

        vv = P[i]
        b = 1 / math.dist(vu, vv)
        x_sum += w * (vv[0] + D[u][i] * (vu[0] - vv[0]) * abs(b))
        y_sum += w * (vv[1] + D[u][i] * (vu[1] - vv[1]) * abs(b))

    return np.array([x_sum / weight_sum, y_sum / weight_sum])

In [ ]:
def change_positions(W, D, P):
    for i in range(len(P)):
        next_p = next_position(i, W, D, P)
        P[i] = next_p
    return P

In [ ]:
G = nx.karate_club_graph()
# G = get_webcola_graph_data()
P, D, W = get_graph_property(G)

plot_graph(G, P)

stress_value = stress(P, D, W)
print(f"{stress_value=}")
while True:
    change_positions(W, D, P)

    next_stress = stress(P, D, W)
    if abs(stress_value - next_stress) < 0.001:
        break

    stress_value = next_stress
    print(f"\r{stress_value=}", end="")

plot_graph(G, P)
plot_spring_layout(G)

## general radial constraints


In [ ]:
def diameter(D):
    return max([max(di) for di in D])

In [ ]:
def radii(u, C, diam):
    min_c = min(C)
    max_c = max(C)
    offset = 0.00000001
    rate = 1 - (C[u] - min_c) / (max_c - min_c + offset)
    return diam * rate / 2

In [ ]:
def calc_closeness_centrality(u, D):
    sum_d = sum([d for d in D[u]])
    return 1 / sum_d if not sum_d < 0.000_001 else 0

In [ ]:
def next_position_general(u, t, W, D, P, R, Z):
    under = sum([(1 - t) * w + t * Z[u][-1]
                for i, w in enumerate(W[u]) if not i == u])

    norm = math.sqrt(P[u][0] ** 2 + P[u][1] ** 2)
    a = 1 / norm if not norm < 0.000_001 else 0
    upper_right_x = t * (R[u] * P[u][0] * a) * Z[u][-1]
    upper_right_y = t * (R[u] * P[u][1] * a) * Z[u][-1]

    if under < 0.000_001:
        return P[u]

    vu = P[u]
    x_sum = 0
    y_sum = 0
    for v in range(len(P)):
        if v == u:
            continue

        vv = P[v]
        d = math.dist(vu, vv)
        b = 1 / d if not d < 0.000_001 else 0
        x_sum += (1 - t) * W[u][v] * (
            vv[0] + D[u][v] * (vu[0] - vv[0]) * abs(b)
        ) + upper_right_x
        y_sum += (1 - t) * W[u][v] * (
            vv[1] + D[u][v] * (vu[1] - vv[1]) * abs(b)
        ) + upper_right_y

    return np.array([x_sum / under, y_sum / under])

In [ ]:
def change_positions_general(t, W, D, P, R, Z):
    for i in range(len(P)):
        next_p = next_position_general(i, t, W, D, P, R, Z)
        P[i] = next_p
    return P

In [ ]:
import networkx as nx
import numpy as np

G = nx.karate_club_graph()
# G = get_football_graph_data()
# G = get_webcola_graph_data()

P, D, W = get_graph_property(G)
nodes = list(G.nodes())
edges = list(G.edges())
print(edges)
P = {i: P[nodes[i]] for i in range(len(nodes))}


C = [calc_closeness_centrality(u, D) for u in range(len(P))]

diam = diameter(D)
R = [radii(i, C, diam) for i in range(len(P))]

D = [np.append(D[i], r) for i, r in enumerate(R)]
D.append(R + [0])

W = [np.append(row, 0) for row in W]
W.append([0] * len(W[0]))

Z = np.zeros_like(W)
for i in range(len(R)):
    r = 1 / (R[i] * R[i]) if R[i] > 0.000_001 else 0
    Z[i][-1] = r
    Z[-1][i] = r


# plot_graph(G, P)

options = {"node_color": "black", "node_size": 50, "linewidths": 0, "width": 0.1}

fig = plt.figure(figsize=(6, 6))
ax = plt.axes()

P = {nodes[i]: P[i] for i in range(len(nodes))}
plot_graph(G, P, options)
P = {i: P[nodes[i]] for i in range(len(nodes))}

k = 100
stress_value = stress(P, D, W)
print(f"{stress_value=}")

for i in range(k + 1):
    change_positions_general(i / k if not i == 0 else 0, W, D, P, R, Z)
    stress_value = stress(P, D, W)
    print(f"\r{stress_value=}", end="")
print()


fig = plt.figure(figsize=(6,6))
ax = plt.axes()
step = 0.25
rad = 0.125
for i in range(10):
    dc = plt.Circle((0, 0), radius=rad, fc="none", ec="red", fill=False, alpha=0.5)
    ax.add_patch(dc)
    rad += step

P = {nodes[i]: P[i] for i in range(len(nodes))}
plot_graph(G, P, options)
P = {i: P[nodes[i]] for i in range(len(nodes))}

## networkx 

- https://networkx.org/documentation/stable/auto_examples/graphviz_layout/plot_lanl_routes.html#sphx-glr-auto-examples-graphviz-layout-plot-lanl-routes-py

> This example needs Graphviz and PyGraphviz.

graphvis
- https://pygraphviz.github.io/documentation/stable/install.html
- > sudo apt-get install graphviz graphviz-dev
- > pip install pygraphviz

In [ ]:
# G = get_webcola_graph_data()
G = nx.karate_club_graph()
pos = nx.nx_agraph.graphviz_layout(G, prog="twopi", root=0)
options = {"node_color": "black", "node_size": 50,
           "linewidths": 0, "width": 0.1}
nx.draw(G, pos, **options)
plt.show()